In [81]:
from __future__ import division
%matplotlib inline
import os
import torch
from torch import nn
import torchvision.transforms.functional as tf
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pdb
import cv2
import random
import glob
import imageio

In [82]:

from unet_tile_se_norm import UNetTileSENorm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
# unet_tile_se_inter
# from utils import np2tensor

class OPT():
    pass

def np2tensor(numpy_array):
    tensor = torch.from_numpy(np.transpose(numpy_array.copy(), (2, 0, 1))).float()/255.*2. - 1
    return tensor.unsqueeze(0)

def tensor2im(input_image, imtype=np.uint8):
    """"Converts a Tensor array into a numpy image array.

    Parameters:
        input_image (tensor) --  the input image tensor array
        imtype (type)        --  the desired type of the converted numpy array
    """
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):  # get the data from a variable
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor.float().numpy()  # convert it into a numpy array
        if image_numpy.shape[0] == 1:  # grayscale to RGB
            image_numpy = np.tile(image_numpy, (3, 1, 1))
        image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0  # post-processing: tranpose and scaling
        image_numpy = np.maximum(image_numpy, 0)
        image_numpy = np.minimum(image_numpy, 255)
    else:  # if it is a numpy array, do nothing
        image_numpy = input_image
    return image_numpy.astype(imtype)

import torchvision.transforms as transforms

def get_transform(opt, params=None, grayscale=False, method=Image.BICUBIC, convert=True):
    transform_list = []
    if grayscale:
        transform_list.append(transforms.Grayscale(1))
    if 'resize' in opt.preprocess:
        osize = [opt.load_size, opt.load_size]
        transform_list.append(transforms.Resize(osize, method))
    elif 'scale_width' in opt.preprocess:
        transform_list.append(transforms.Lambda(lambda img: __scale_width(img, opt.load_size, method)))

    if 'crop' in opt.preprocess:
        if params is None:
            transform_list.append(transforms.RandomCrop(opt.crop_size))
        else:
            transform_list.append(transforms.Lambda(lambda img: __crop(img, params['crop_pos'], opt.crop_size)))

    if opt.preprocess == 'none':
        pass
#         transform_list.append(transforms.Lambda(lambda img: __make_power_2(img, base=4, method=method)))

    if not opt.no_flip:
        if params is None:
            transform_list.append(transforms.RandomHorizontalFlip())
        elif params['flip']:
            transform_list.append(transforms.Lambda(lambda img: __flip(img, params['flip'])))

    if convert:
        transform_list += [transforms.ToTensor()]
        if grayscale:
            transform_list += [transforms.Normalize((0.5,), (0.5,))]
        else:
            transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    return transforms.Compose(transform_list)




In [87]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import OrderedDict
import pdb
import colour
from colour.plotting import *
import imageio
from colour_checker_detection import (
    EXAMPLES_RESOURCES_DIRECTORY,
    colour_checkers_coordinates_segmentation,
    detect_colour_checkers_segmentation)

from colour_checker_detection.detection.segmentation import (
    adjust_image)

colour.utilities.describe_environment();
colour_style();




fore_list = glob.glob('./portrait_image/*_cloud_input_WB.jpg') + glob.glob('./portrait2/*_cloud_input_WB.jpg')
back_list = glob.glob('./street_image/*.jpg') + glob.glob('./background_image/*.jpg')+ glob.glob('./background2/*.jpg')
# back_list = glob.glob('./portrait2/*_cloud_input_WB.jpg') + glob.glob('./portrait_image/*_cloud_input_WB.jpg') + glob.glob('./street_image/*_WB.jpg') + glob.glob('./background_image/*_WB.jpg')

def fitting(fore_name, fore_path, dir_path, example_name, mask_0, back_image_0):
    EXAMPLES_RESOURCES_DIRECTORY = dir_path
    COLOUR_CHECKER_IMAGE_PATHS = glob.glob(
        os.path.join(EXAMPLES_RESOURCES_DIRECTORY, '*.jpg'))
    COLOUR_CHECKER_IMAGES = [
        colour.cctf_decoding(colour.io.read_image(path))
        for path in COLOUR_CHECKER_IMAGE_PATHS
    ]

    for iteration in range(1):

        color_0 = cv2.resize(COLOUR_CHECKER_IMAGES[1], (64,64)).reshape((-1,3))
        color_1 = cv2.resize(COLOUR_CHECKER_IMAGES[0], (64,64)).reshape((-1,3))
        output = colour.cctf_encoding(
                colour.colour_correction(
                    colour.cctf_decoding(colour.io.read_image(fore_path)), color_0, color_1, terms = 17))
        output = np.clip(output, 0, 1)

        comp = output*(mask_0) + back_image_0 * (1-mask_0)

        imageio.imwrite('./dovenet_compare_psnr_lr/'+ fore_name+example_name,input_image*255)
        
def process(inp_img, ref_img, inp_show, model_1, dir_path):
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)
    SS_scratch_output, _, _ = model_1(inp_img, ref_img)
    SS_scratch_show = tensor2im(SS_scratch_output[0])
    SS_scratch_show = SS_scratch_show.astype(float)/255
    Image.fromarray(np.uint8(inp_show*255)).save("./" + dir_path + "/input.jpg")
    Image.fromarray(np.uint8(SS_scratch_show*255)).save("./" + dir_path + "/result.jpg")

image_small_list = []
# for root, dirs, files in os.walk("../16_fore_mask/"):
#     for file in files:
#         if "small" in file:
#             image_small_list.append(os.path.join(root, file))
for root, dirs, files in os.walk("../camera_ready/vendor_testing_1/"):
    for file in files:
        if "small" in file:
            image_small_list.append(os.path.join(root, file))
for root, dirs, files in os.walk("../camera_ready/vendor_testing_2/"):
    for file in files:
        if "small" in file:
            image_small_list.append(os.path.join(root, file))
            
for root, dirs, files in os.walk("../camera_ready/vendor_testing_3/"):
    for file in files:
        if "small" in file:
            image_small_list.append(os.path.join(root, file))
            
print(len(image_small_list))
for index, file in enumerate(image_small_list):
    if 1 == 1:
        print(index)
        fore_path = file
        fore_shift_path = file.replace("_small.jpg", ".jpg")
        mask_path = file.replace("_small.jpg", "_mask.jpg")
        bg_path = file.replace("_small.jpg", "_fore.jpg")
        gt_path = file.replace("_small.jpg", "_gt.jpg")
        fore_name = bg_path.replace("_fore.jpg", "").replace("../vendor_testing_2/", "").replace("../vendor_testing_1/", "").replace("../vendor_testing_3/", "")
        fore_name = fore_name.split("/")[-1]
        if os.path.isfile('./output_1/'+ fore_name+'_ssh.jpg'):
            continue
#         fore_name = fore_path.split('/')[-1][:-4]
#         bg_name = bg_path.split('/')[-1][:-4]
#         high_res_path = './final_result/' + fore_name + '/' + bg_name + '.jpg'
        inp_img = Image.open(fore_path).convert('RGB')
        inp_shift_img = Image.open(fore_shift_path).convert('RGB')
        ref_img = Image.open(bg_path).convert('RGB')
        mask_img = Image.open(mask_path).convert('RGB')
        gt_img = Image.open(gt_path).convert('RGB')

        inp_img = tf.resize(inp_img, [256, 256])
        inp_shift_img = tf.resize(inp_shift_img, [256, 256])
        ref_img = tf.resize(ref_img, [256, 256])
        mask_img = tf.resize(mask_img, [256, 256])
        gt_img = tf.resize(gt_img, [256, 256])
        
        inp_img = np.array(inp_img)
        inp_shift_img = np.array(inp_shift_img)
        ref_img = np.array(ref_img)
        mask_img = np.array(mask_img)
        gt_img = np.array(gt_img)

        inp_img = np2tensor(inp_img)
        inp_shift_img = np2tensor(inp_shift_img)
        ref_img = np2tensor(ref_img)
        mask_img = np2tensor(mask_img)

#         ratio = 0.95
#         output, _, style = model_inter(inp_img, ref_img, inp_img, ratio)

        inp_show = tensor2im(inp_img[0])
        inp_shift_show = tensor2im(inp_shift_img[0])
        ref_show = tensor2im(ref_img[0])
#         oup_show = tensor2im(output[0])

        # import pdb
        mask_img1 = tensor2im(mask_img.squeeze())
        mask_img1 = mask_img1.astype(float)/255
        # pdb.set_trace()

#         oup_show = oup_show.astype(float)/255
        ref_show = ref_show.astype(float)/255
        inp_show = inp_show.astype(float)/255
        inp_shift_show = inp_shift_show.astype(float)/255

#         comp_our = oup_show*(mask_img1)+ref_show*(1-mask_img1)
        comp_input = inp_shift_show*(mask_img1)+ref_show*(1-mask_img1)

        mask_img1 = mask_img1[:,:,0]
        comp = Image.fromarray(np.uint8(comp_input*255))
        mask = Image.fromarray(np.uint8(mask_img1*255))
        real = Image.fromarray(np.uint8(comp_input*255))

        # apply the same transform to composite and real images
        comp = transform(comp)
        mask = tf.to_tensor(mask)
        real = transform(real)
        # concate the composite and mask as the input of generator
        inputs=torch.cat([comp,mask],0)

#         dove_output = DoveNet_model(inputs.unsqueeze(0))
        mask = mask.unsqueeze(0)
        input_show = tensor2im(comp)
#         output_show = tensor2im(dove_output[0])

        mask_img1_3 = np.zeros_like(comp_input)
        mask_img1_3[:,:,0] = mask_img1
        mask_img1_3[:,:,1] = mask_img1
        mask_img1_3[:,:,2] = mask_img1







        input_image = np.array(Image.open(fore_shift_path).convert('RGB')).astype(float)/255
        back_image = np.array(Image.open(bg_path).convert('RGB')).astype(float)/255
        mask = np.array(Image.open(mask_path).convert('RGB')).astype(float)

        mask3 = np.zeros_like(input_image)
        mask3[:,:,0] = mask[:,:,0]/255
        mask3[:,:,1] = mask[:,:,0]/255
        mask3[:,:,2] = mask[:,:,0]/255  
        print(fore_name)         
            
        checkpoint = torch.load("./checkpoint", map_location='cpu')
        opt.norm = "BN"
        opt.style_norm = "BN"
        model_1 = UNetTileSENorm(opt)
        model_1.load_state_dict(checkpoint['model_state_dict'])
        model_1.eval()
        process(inp_img, ref_img, inp_show, model_1, "NEW_SS_transform_2")
#             fitting(fore_name, fore_path, 'NEW_SS_transform_1', '_64_128_cycle0.1_newcube.jpg', mask3, back_image)
        EXAMPLES_RESOURCES_DIRECTORY = 'NEW_SS_transform_2'
        COLOUR_CHECKER_IMAGE_PATHS = glob.glob(os.path.join(EXAMPLES_RESOURCES_DIRECTORY, '*.jpg'))
        COLOUR_CHECKER_IMAGES = [
            colour.cctf_decoding(colour.io.read_image(path))
            for path in COLOUR_CHECKER_IMAGE_PATHS
        ]

        for iteration in range(1):

            color_0 = cv2.resize(COLOUR_CHECKER_IMAGES[1], (64,64)).reshape((-1,3))
            color_1 = cv2.resize(COLOUR_CHECKER_IMAGES[0], (64,64)).reshape((-1,3))
            output = colour.cctf_encoding(
                    colour.colour_correction(
                        colour.cctf_decoding(colour.io.read_image(fore_shift_path)), color_0, color_1, terms = 17))
            output = np.clip(output, 0, 1)

            comp = output[:, :, :3]*(mask3) + back_image * (1-mask3)
            comp = output[:, :, :3]*(mask3) + back_image * (1-mask3)
            comp = comp*255
            comp = Image.fromarray(np.uint8(comp))
            comp.save('./output_1/'+ fore_name+'_ssh.jpg')
        

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31)     *
*                [GCC 4.2.1 Compatible Clang 4.0.1                            *
*   (tags/RELEASE_401/final)]                                                 *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : 0.3.15                                                       *
*       colour-checker-detection : 0.1.1                                      *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.9.0                                                       *
*       matplotlib : 3.3.0              

/Users/yifanjiang/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/Users/yifanjiang/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/Users/yifanjiang/anaconda3/envs/torch/lib/python3.6/site-packages/colour/utilities/verbose.py:235: ColourUsageWarning: "OpenImageIO" related API features are not available, switching to "Imageio"!
  warn(*args, **kwargs)


201
desert-2576929_gt920
202
condominium-690086
203
winter-2080070
204
beach-768421
205
city-1150026
206
forest-3394066
207
mountains-440520
208
seascape-4636264
209
vaulted-cellar-247391
210
ameland-5651866
211
downtown-690827
212
beach-1236581
213
alley-3453557
214
room-597166
215
benches-560435
216
neighborhood-802074
217
beach-5622187
218
ocean-931776
219
arches-1850730


In [88]:
import os
import cv2

In [89]:
from skimage.measure import compare_mse as mse
from skimage.measure import compare_psnr as psnr
from skimage.measure import compare_ssim as ssim

total_psnr = 0
total_ssim = 0
total_mse = 0
total_lpips = 0
index_1 = 0
for files, dirs, root in os.walk("./output_1"):
    for file in root:
        if ".jpg" in file:
#             print(file)
            ssh = cv2.imread("./output_1/"+file)
            for name in image_small_list:
                if file.replace("_ssh.jpg", "") in name:
#                     print(name)
                    index_1 += 1
                    gt = cv2.imread(name.replace("_small_fore", "_small").replace("_small", "_gt"))
                    gt = cv2.resize(gt, (256,256), interpolation=cv2.INTER_AREA)
                    ssh = cv2.resize(ssh, (256,256), interpolation=cv2.INTER_AREA)
                    psnr_1 = psnr(gt, ssh, data_range=ssh.max() - ssh.min())
                    
                    total_psnr += psnr_1
# print(len(image_small_list))
print(index_1)
print(total_psnr/index_1)
                    
#             img = cv2.imread(f"../camera_ready/vendor_testing_3/{file}")
#             cv2.imwrite(f"../camera_ready/vendor_testing_3/{file}", img)
#             replace = file.replace(".png", ".jpg")
#             os.system(f"mv ../camera_ready/vendor_testing_3/{file} ../camera_ready/vendor_testing_3/{replace}")
#             print(f"mv ../camera_ready/vendor_testing_3/{file} ../camera_ready/vendor_testing_3/{replace}")

/Users/yifanjiang/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.


216
28.0443528771


In [1]:
index_1
total_psnr/200

NameError: name 'index_1' is not defined